# This notebook has the functions necessary to generate the input file used in the BEE_SNN and also read/translate the output file generated by the simulator

In [1]:
import numpy

In [2]:
# Supposing a network with a total of "SpkLiq_number_of_neurons" neurons:
SpkLiq_number_of_neurons=15*30*36

# Each byte has 8 bits, so a long int has 64bits. The system must always round up
SpkLiq_number_of_long_ints = (SpkLiq_number_of_neurons>>6)+((SpkLiq_number_of_neurons&(64-1))>0)
SpkLiq_test_vthres_bits = numpy.zeros(SpkLiq_number_of_long_ints,dtype=numpy.int64)

# The number of long ints (8 bytes) necessary to encode the spikes using only '1's and '0's is:
SpkLiq_number_of_long_ints

254

In [25]:
# This function encodes the spikes using bits to represent the neurons in a sequence of long integers
def set_input_neuron_bits(SpkLiq_number_of_neurons,input_list):
    '''
    SpkLiq_number_of_neurons: number of neurons
    input_list: list with the indices of the neurons that should spike (value 1)
    '''
    SpkLiq_number_of_long_ints = (SpkLiq_number_of_neurons>>6)+((SpkLiq_number_of_neurons&(64-1))>0)
    SpkLiq_test_vthres_bits = numpy.zeros(SpkLiq_number_of_long_ints,dtype=numpy.int64)
    one_int64 = numpy.array([1],dtype=numpy.int64)[0]
    for i in input_list:
        SpkLiq_test_vthres_bits[i>>6] |= (one_int64<<(i&(64-1)));    
#     for i in input_list:
#         SpkLiq_test_vthres_bits[i>>6] |= (1<<(i&(64-1)));
    return SpkLiq_test_vthres_bits

In [27]:
# Testing:
set_input_neuron_bits(SpkLiq_number_of_neurons,[14783,3,65])
# 1<<(14783&(64-1))

array([                   8,                    2,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0,                    0,
                          0,                    0, 

In [5]:
# This function generates a file full of random spikes according to the network size
def generate_random_file(filename,number_of_iteractions,number_of_neurons_liquid,number_of_neurons_input):
    
    input_list = numpy.arange(number_of_neurons_liquid)
    
    with open(filename, "wb") as f:
        for i in xrange(number_of_iteractions):
            numpy.random.shuffle(input_list)
            output = set_input_neuron_bits(number_of_neurons_liquid,input_list[:number_of_neurons_input])
            output.tofile(f)

In [6]:
# generate_random_file("test_sim_exc_inputs.bin",1000,15*30*30,15*30*30)
# generate_random_file("mega_sim_exc_inputs.bin",10,300*60*60,300*60*60/2)

In [7]:
# So far my simulator is only working with fixed weights to the inputs.
# One reason for this is that if one is using population coding, the weights should not be changed (???).
# Another reason is a more practical one: weights are, at least, a 4 bytes value, so in a long simulation with a 
# big network the file size would became impractical very fast (one million neurons would generate 4MB per step!)
def generate_weights_file(filename, weights_array):
    output = numpy.array(weights_array, dtype=numpy.float32)
    with open(filename, "wb") as f:
            output.tofile(f)

In [8]:
# The input weights file MUST have the same size as the base name used with the simulation
# generate_weights_file("test_sim_exc_inputs_weights.bin", [10E-9]*(15*30*30))
# generate_weights_file("small_sim_exc_inputs_weights.bin", [10000E-9]*(10*2*2))

In [10]:
# This function generates the input file according to the input MATRIX supplied.
def generate_file(filename,number_of_neurons_liquid,input_matrix):
    with open(filename, "wb") as f:
        for i in xrange(len(input_matrix)):
            output = set_input_neuron_bits(number_of_neurons_liquid,input_matrix[i])
            output.tofile(f)

In [40]:
# Testing:
generate_file("small_sim_exc_inputs.bin",10*2*2,[range(10*2*2)])

In [11]:
# This function reads an array encoded using the same scheme as set_input_neuron_bits
# but prints the results in an user friendly way.
# It's only useful for testing purposes..
def generate_input_list_from_bits_old(input_bytes):
    test_bits = numpy.arange(64,dtype=numpy.int64)
    count = 0
    for i in input_bytes:
        for j in test_bits:
            if (i & (1<<(j&(64-1)))):
                print count*64+j,bin(i & (1<<(j&(64-1))))
        count+=1

In [12]:
# Testing:
generate_input_list_from_bits_old(set_input_neuron_bits(200,[0,1,32]))

0 0b1
1 0b10
32 0b100000000000000000000000000000000


In [13]:
# This function reads an array encoded using the same scheme as set_input_neuron_bits
# and returns a numpy array with the indices of the neurons who spiked
def generate_list_from_bits(SpkLiq_number_of_neurons,input_bytes):
    output = numpy.empty(SpkLiq_number_of_neurons,dtype=numpy.int32)
    test_bits = numpy.arange(64,dtype=numpy.int64)
    count=0
    output_idx=0
    for i in input_bytes:
        for j in test_bits:
            result = (1<<(j&(64-1)))
            if (i & result):
                output[output_idx]=count*64+j
                output_idx+=1
        count+=1
    return output[:output_idx]

In [14]:
# Testing:
generate_list_from_bits(135,set_input_neuron_bits(135,[165,32]))

array([ 32, 165], dtype=int32)

In [15]:
# Reads the output file generated by the simulator or the input file
# and returns a numpy array where each row represents the output/input of one simulation step
# Only the indices of the neurons indicated with spike ('1') are returned.
def read_file(sim_file, SpkLiq_number_of_neurons, number_of_steps):
    SpkLiq_number_of_long_ints = (SpkLiq_number_of_neurons>>6)+((SpkLiq_number_of_neurons&(64-1))>0) 
    with open(sim_file, "rb") as f:
        return numpy.split(numpy.fromfile(f,dtype=numpy.int64,count=SpkLiq_number_of_long_ints*number_of_steps),number_of_steps)

In [16]:
NofN = 15*30*36
simulated_steps = 10
output = read_file("rdc2_sim_outputs.bin", NofN, simulated_steps)
for i in xrange(simulated_steps):
    print "i:",i,"=>",generate_list_from_bits(NofN,output[i])

i: 0 => [  706  1362  9423 10507 16090]
i: 1 => []
i: 2 => [ 7001  8985 14737 15424]
i: 3 => [ 2143  2144  2145  2146  2147  2148  2149  2150  2151  2152  2153  2154
  2155  2156  2157  2158  2159  2160  2161  2162  2163  2164  2165  2166
  2167  2168  2169  2170  2171  2172  2173  2174  2175  7884  9606 12317
 14359]
i: 4 => [ 8321 11807 11808 11809 11810 11811 11812 11813 11814 11815 11816 11817
 11818 11819 11820 11821 11822 11823 11824 11825 11826 11827 11828 11829
 11830 11831 11832 11833 11834 11835 11836 11837 11838 11839 14480 15891]
i: 5 => [ 3218  6272  9096 13653]
i: 6 => [ 4876  6592  8338 10836 12228 12382 13259 14033]
i: 7 => [ 7301  8128  8588 10012 12300 12429 13184 13721 13838 14537 15005 15443
 15961]
i: 8 => [ 5701  7312  8973  9923 10322 10825 10969 12226 14086 15965]
i: 9 => [ 9436  9670  9689  9868 10009 10051 10053 10055 10881 10882 11917 12184
 12255 12256 12257 12258 12259 12260 12261 12262 12263 12264 12265 12266
 12267 12268 12269 12270 12271 12272 12273 1227